In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

In [2]:
STATIONS = ['S32', 'S33', 'S34']
train_date_part = pd.read_csv('input_orig/train_date.csv', nrows=10000)
date_cols = train_date_part.drop('Id', axis=1).count().reset_index().sort_values(by=0, ascending=False)
date_cols['station'] = date_cols['index'].apply(lambda s: s.split('_')[1])
date_cols = date_cols[date_cols['station'].isin(STATIONS)]
date_cols = date_cols.drop_duplicates('station', keep='first')['index'].tolist()
print(date_cols)
train_date = pd.read_csv('input_orig/train_date.csv', usecols=['Id'] + date_cols)
print(train_date.columns)
test_date = pd.read_csv('input_orig/test_date.csv', usecols=['Id'] + date_cols)
print(test_date.columns)
train_date.columns = ['Id'] + STATIONS
test_date.columns = ['Id'] + STATIONS
for station in STATIONS:
    train_date[station] = 1 * (train_date[station] >= 0)
    test_date[station] = 1 * (test_date[station] >= 0)
response = pd.read_csv('input_orig/train_numeric.csv', usecols=['Id', 'Response'])
print(response.shape)
train = response.merge(train_date, how='left', on='Id')
# print(train.count())
train.head(3)

['L3_S34_D3877', 'L3_S33_D3870', 'L3_S32_D3852']
Index(['Id', 'L3_S32_D3852', 'L3_S33_D3870', 'L3_S34_D3877'], dtype='object')
Index(['Id', 'L3_S32_D3852', 'L3_S33_D3870', 'L3_S34_D3877'], dtype='object')
(1183747, 2)


,Id,Response,S32,S33,S34
0,4,0,0,1,1
1,6,0,0,1,1
2,7,0,0,1,1


In [5]:
test = test_date
test.head()

,Id,S32,S33,S34
0,1,0,1,1
1,2,0,1,1
2,3,0,1,1
3,5,0,1,1
4,8,0,1,1


In [6]:
train['cnt'] = 1
test['cnt'] = 1
failure_rate = train.groupby(STATIONS).sum()[['Response', 'cnt']]
failure_rate['failure_rate'] = failure_rate['Response'] / failure_rate['cnt']
failure_rate = failure_rate[failure_rate['cnt'] > 1000]  # remove 
failure_rate.head(20)

Response      cnt  failure_rate
S32 S33 S34                                 
0   0   0         319    62482      0.005105
    1   0           5     1529      0.003270
        1        5449  1095190      0.004975
1   0   0         837     4602      0.181877
        1         177     1965      0.090076
    1   1          92    17960      0.005122

In [7]:
train_date_part = pd.read_csv('input_orig/train_date.csv', nrows=10000)

In [8]:
date_cols = train_date_part.drop('Id', axis=1).count().reset_index().sort_values(by=0, ascending=False)

In [9]:
date_cols['station'] = date_cols['index'].apply(lambda s: s.split('_')[1])
date_cols = date_cols.drop_duplicates('station', keep='first')['index'].tolist()

In [10]:
# Train start dates
train_start_date = pd.read_csv('input_orig/train_date.csv', usecols=['Id'] + date_cols)
train_start_date['start_date'] = train_start_date[date_cols].min(axis=1)
train_start_date = train_start_date.drop(date_cols, axis=1)
print(train_start_date.shape)

(1183747, 2)


In [11]:
# Test start dates
test_start_date = pd.read_csv('input_orig/test_date.csv', usecols=['Id'] + date_cols)
test_start_date['start_date'] = test_start_date[date_cols].min(axis=1)
test_start_date = test_start_date.drop(date_cols, axis=1)
print(test_start_date.shape)

(1183748, 2)


In [12]:
start_date = pd.concat([train_start_date, test_start_date])
print(start_date.shape)

(2367495, 2)


In [13]:
del train_start_date, test_start_date
gc.collect()
start_date.head()

,Id,start_date
0,4,82.24
1,6,1313.12
2,7,1618.70
3,9,1149.20
4,11,602.64


In [14]:
train_id = pd.read_csv('input_orig/train_numeric.csv', usecols=['Id'])
test_id = pd.read_csv('input_orig/test_numeric.csv', usecols=['Id'])
train_id = train_id.merge(start_date, on='Id')
test_id = test_id.merge(start_date, on='Id')
train_test_id = pd.concat((train_id, test_id)).reset_index(drop=True).reset_index(drop=False)
train_test_id = train_test_id.sort_values(by=['start_date', 'Id'], ascending=True)
train_test_id['IdDiff1'] = train_test_id['Id'].diff().fillna(9999999).astype(int)
train_test_id['IdDiff2'] = train_test_id['Id'].iloc[::-1].diff().fillna(9999999).astype(int)
train_test_id['Magic'] = 1 + 2 * (train_test_id['IdDiff1'] > 1) + 1 * (train_test_id['IdDiff2'] < -1)

train_with_magic = train.merge(train_test_id[['Id', 'Magic']], on='Id')
test_with_magic = test.merge(train_test_id[['Id', 'Magic']], on='Id')
train_with_magic.head()

,Id,Response,S32,S33,S34,cnt,Magic
0,4,0,0,1,1,1,2
1,6,0,0,1,1,1,2
2,7,0,0,1,1,1,2
3,9,0,0,1,1,1,2
4,11,0,0,1,1,1,1


In [15]:
test_with_magic.head()

,Id,S32,S33,S34,cnt,Magic
0,1,0,1,1,1,2
1,2,0,1,1,1,2
2,3,0,1,1,1,2
3,5,0,1,1,1,2
4,8,0,1,1,1,2


In [16]:
magic_failure_rate = train_with_magic.groupby(['Magic']).sum()[['Response', 'cnt']]
magic_failure_rate['failure_rate'] = magic_failure_rate['Response'] / magic_failure_rate['cnt']
magic_failure_rate.head()

,Response,cnt,failure_rate
Magic,,,
1,971,22221,0.043697
2,142,82312,0.001725
3,4334,82159,0.052751
4,1432,997055,0.001436


In [17]:
combined_failure_rate = train_with_magic.groupby(['Magic','S32', 'S33']).sum()[['Response', 'cnt']]
combined_failure_rate['failure_rate'] = combined_failure_rate['Response'] / combined_failure_rate['cnt']
combined_failure_rate.head(20)

Response     cnt  failure_rate
Magic S32 S33                                
1     0   0          39    1367      0.028530
          1         707   20159      0.035071
      1   0         218     378      0.576720
          1           7     317      0.022082
2     0   0           9    4820      0.001867
          1         118   75111      0.001571
      1   0          12    1089      0.011019
          1           3    1292      0.002322
3     0   0         225    4773      0.047140
          1        3358   75030      0.044755
      1   0         698    1009      0.691774
          1          53    1347      0.039347
4     0   0          46   51525      0.000893
          1        1271  926419      0.001372
      1   0          86    4091      0.021022
          1          29   15020      0.001931

In [22]:
mag_tr_69 = train_with_magic[(train_with_magic.Magic==3)&(train_with_magic.S32==1)&(train_with_magic.S33==0)]
mag_te_69 = test_with_magic[(test_with_magic.Magic==3)&(test_with_magic.S32==1)&(test_with_magic.S33==0)]

mag_tr_57 = train_with_magic[(train_with_magic.Magic==1)&(train_with_magic.S32==1)&(train_with_magic.S33==0)]
mag_te_57 = test_with_magic[(test_with_magic.Magic==1)&(test_with_magic.S32==1)&(test_with_magic.S33==0)]

In [29]:
mag_tr_69.to_csv('magic_tr_69_beluga.csv', index=False)
mag_te_69.to_csv('magic_te_69_beluga.csv', index=False)

mag_tr_57.to_csv('magic_tr_57_beluga.csv', index=False)
mag_te_57.to_csv('magic_te_57_beluga.csv', index=False)